In [1]:
import os
#os.chdir(r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\zavmo")
os.chdir("/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo")

In [2]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import Markdown

In [3]:
from helpers.agents.common import get_yaml_data,get_agent_instructions

In [4]:
from helpers.swarm import Agent, Response, Result, Tool, run_step, AgentFunction, function_to_json
from helpers.agents.common import get_agent_instructions

In [5]:
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall, Function

In [6]:
import json

In [23]:
from pydantic import BaseModel, Field
from typing import List, Dict

In [56]:
class request_curriculum(Tool):
    """Request a curriculum from the curriculum design team."""
    instructions: str = Field(description="Instructions for generating the curriculum")
    learning_objectives: str = Field(description="The learner's learning objectives")
    interest_areas: str = Field(description="The learner's interest areas")
    time_available: str = Field(description="The time available for the learner to study per week")
    
    def execute(self, context: Dict):
        print(f"Curriculum request received for {context['email']} with sequence id {context['sequence_id']}")
        conversation_summary = summarize_history(context['history'])
        user_prompt = f"Here is a summary of the conversation so far: {conversation_summary}\n\n"
        user_prompt += f"Learning objectives: {self.learning_objectives}\n\n"
        user_prompt += f"Interest areas: {self.interest_areas}\n\n"
        user_prompt += f"Time available per week: {self.time_available}\n\n"
        user_prompt += f"Instructions: {self.instructions}"
        
        messages = [
            {"role": "system", "content": get_prompt('curriculum')},
            {"role": "user", "content": user_prompt}
        ]
        openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        response = openai_client.beta.chat.completions.parse(
            model="gpt-4o",
            messages=messages,
            response_format=Curriculum
        )
        model_result = response.choices[0].message.parsed
        
        context['stage_data']['discuss']['curriculum'] = model_result.model_dump()
        curriculum_text = str(model_result)
        value = f"Curriculum generated successfully:\n\n{curriculum_text}"
        return Result(value=value, context=context)

In [57]:
import openai

In [58]:
request_curriculum.__name__

'request_curriculum'

In [59]:
import inspect

In [ ]:
from helpers.agents.a_discover import discover_agent, update_discover_data
from helpers.agents.b_discuss import discuss_agent, request_curriculum

In [ ]:
active_agent   = discuss_agent
messages       = []
context        = {"email":"aditya@gmail.com","sequence_id":'52442',
                  "stage_data":{'profile':{},'discuss':{},'delivery':{},'demonstrate':{}},
                  "stage":1}

In [ ]:
result = run_step(active_agent, messages, context=context, max_turns=5)

messages.extend(result.messages)
context.update(result.context)
if result.agent!=active_agent:
    active_agent = result.agent
    context['stage'] += 1
    print(f"Switching agent to {active_agent.name}")
print(context)

Markdown(messages[-1]['content'])

In [ ]:
user_resp = "Sounds good, let's go"

messages.append({"role":"user","content":user_resp})